In [3]:
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback

import wandb

# Load a model
model = YOLO(
    "weights/yolov8n-seg.pt", task="segment"
)  # load a pretrained model (recommended for training)

yaml_file = "configs/fashion_five.yml"
project_name = "human_parsing"
add_wandb_callback(model)

# Train the model
results = model.train(project=project_name, data=yaml_file, epochs=100, imgsz=1024)

# Finish the W&B run
wandb.finish()

New https://pypi.org/project/ultralytics/8.2.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.7 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (GRID A100D-2-20C MIG 2g.18gb, 20476MiB)
engine/trainer: task=segment, mode=train, model=weights/yolov8n-seg.pt, data=configs/fashion_five.yml, epochs=100, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=human_parsing, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

train: Scanning /home/jordan/SEGMENT/datasets/fashion_five/labels/train... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<00:00, 658.93it/s]

train: New cache created: /home/jordan/SEGMENT/datasets/fashion_five/labels/train.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/jordan/SEGMENT/datasets/fashion_five/labels/val... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<00:00, 618.36it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 30.55it/s]


                   all          1          6     0.0224        0.4      0.298      0.119     0.0133        0.2     0.0995     0.0239
                  hair          1          1     0.0455          1      0.995      0.398          0          0          0          0
                  face          1          1          0          0          0          0          0          0          0          0
                  neck          1          1          0          0          0          0          0          0          0          0
                  hand          1          2          0          0          0          0          0          0          0          0
                person          1          1     0.0667          1      0.497      0.199     0.0667          1      0.497      0.119


Generating Visualizations for batch-1/1:   0%|          | 0/1 [00:00<?, ?it/s]

Speed: 0.7ms preprocess, 8.6ms inference, 0.0ms loss, 8.0ms postprocess per image
Results saved to human_parsing/train


lr/pg0,▁▂▂▃▃▄▅▅▅▆▆▆▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▂▂
lr/pg1,▁▂▂▃▃▄▅▅▅▆▆▆▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▂▂
lr/pg2,▁▂▂▃▃▄▅▅▅▆▆▆▇▇▇███████████▇▇▇▇▆▆▅▅▅▄▄▃▂▂
metrics/mAP50(B),▅██▃▂▂▂▃▃▃▃▄▃▃▄▄▄██▄▄▄▄▂▃▁▁▃▃▃▃▃▃▃▃▃▃▃▆█
metrics/mAP50(M),▅█▃▂▂▂▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅
metrics/mAP50-95(B),▅▇█▄▃▃▂▄▆▅▄▄▃▂▂▃▃▆▆▄▃▄▄▃▂▂▁▁▁▁▃▂▂▃▃▂▃▃▄█
metrics/mAP50-95(M),▇█▄▃▂▂▂▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▆▆███▁
metrics/precision(M),▆▆▇▇▇█▇█▁▁▁▁▁▁▁▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆
metrics/recall(B),█████████████████████████████████▁▁▁▁▁▁█
metrics/recall(M),████████▁▁▁▁▁▁▁▁▁▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█


In [ ]:
from ultralytics.data.base import BaseDataset

BaseDataset.get_labels()

In [ ]:
model_path = "runs/segment/train/weights/best.pt"
model = YOLO(model_path, task="segment")

image_dir = "/home/jordan/jd_segment_anything/datasets/person_seg/images/train"
images = [os.path.join(image_dir, file) for file in os.listdir(image_dir)]
image = random.choice(images)
image = Image.open(image)

result = model(image)

In [ ]:
import requests
from PIL import Image

from utils import *

url = "https://fashionista.com/.image/ar_4:3%2Cc_fill%2Ccs_srgb%2Cfl_progressive%2Cq_auto:good%2Cw_1200/MTYxNjk2NzIwOTkwNzc0NTc0/main-anok-yai-versace-leather-dress-spring-2019-milan-fashion-week.jpg"
image = Image.open(requests.get(url, stream=True).raw)
image = resize_preserve_aspect_ratio(image, 1024)
results = model(image)

In [ ]:
import yaml

with open(yaml_file, 'r') as file:
    data = yaml.load(file, Loader=yaml.FullLoader)

labels_dict = data.get('names')

In [ ]:
result = results[0]
labels = result.boxes.cls 
labels = [int(label.item()) for label in labels]
labels = [labels_dict[label] for label in labels]
masks = result.masks.data
masks = [unload_mask(mask) for mask in masks]

In [ ]:
masks_dict = dict(zip(labels, masks))
for k, v in masks_dict.items():
    print(k)
    overlay = overlay_mask(image, v)
    display(overlay)